In [4]:
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns',None)

In [2]:
train = pd.read_parquet('./dataset/train_파생변수.parquet')
test = pd.read_parquet('./dataset/test_파생변수.parquet')

# (요일별 가중치,금요일 여부) Column 추가

In [ ]:
#day_weight 컬럼 추가
train['day_weight'] = 0

train.loc[(train['day_of_week'] == '월')|(train['day_of_week'] == '금'),'day_weight'] = 2
train.loc[(train['day_of_week'] == '토')|(train['day_of_week'] == '일'),'day_weight'] = 3
train.loc[(train['day_of_week'] == '화')|(train['day_of_week'] == '수')|(train['day_of_week'] == '목'),'day_weight'] = 1

test['day_weight'] = 0

test.loc[(test['day_of_week'] == '월')|(test['day_of_week'] == '금'),'day_weight'] = 2
test.loc[(test['day_of_week'] == '토')|(test['day_of_week'] == '일'),'day_weight'] = 3
test.loc[(test['day_of_week'] == '화')|(test['day_of_week'] == '수')|(test['day_of_week'] == '목'),'day_weight'] = 1

train['isfriday']=train.progress_apply(lambda row : True if row['day_of_week']=='금' else False,axis=1)
test['isfriday']=test.progress_apply(lambda row : True if row['day_of_week']=='금' else False,axis=1)

# 구간 길이 파생 변수 추가

In [11]:
# !pip install haversine
from haversine import haversine

# 위도경도를 이용해서 거리 구하기(Km기준)
tqdm.pandas()
train['distance']= train.progress_apply(lambda row : haversine((row['start_latitude'], row['start_longitude']),(row['end_latitude'],row['end_longitude']),unit='km'),axis=1)
test['distance']= test.progress_apply(lambda row : haversine((row['start_latitude'], row['start_longitude']),(row['end_latitude'],row['end_longitude']),unit='km'),axis=1)

100%|██████████| 291241/291241 [00:08<00:00, 33675.10it/s]


# (요일별, 시간별, 도로별) 평균속도 파생변수 추가 (3 columns)

In [ ]:
print(f"Trains : {len(train)} , Tests : {len(test)}")
hour_mean = train.groupby(['base_hour']).mean()
hour_mean.drop(columns=hour_mean.columns.drop('target'), inplace=True)
hour_mean.rename(columns={'target':'hour_mean_speed'},inplace=True)

day_mean = train.groupby(['day_of_week']).mean()
day_mean.drop(columns=day_mean.columns.drop('target'), inplace=True)
day_mean.rename(columns={'target':'day_mean_speed'},inplace=True)

road_mean = train.groupby(['road_name']).mean()
road_mean.drop(columns=road_mean.columns.drop('target'), inplace=True)
road_mean.rename(columns={'target':'road_mean_speed'},inplace=True)


train = pd.merge(train,hour_mean,how="left",on=['base_hour'])
train = pd.merge(train,day_mean,how="left",on=['day_of_week'])
train = pd.merge(train,road_mean,how="left",on=['road_name'])

test = pd.merge(test,hour_mean,how="left",on=['base_hour'])
test = pd.merge(test,day_mean,how="left",on=['day_of_week'])
test = pd.merge(test,road_mean,how="left",on=['road_name'])

print(f"Trains : {len(train)} , Tests : {len(test)}")
print(f"Trains Nan : {train['hour_mean_speed'].isna().sum()},  Test Nan : {test['hour_mean_speed'].isna().sum()}")
print(f"Trains Nan : {train['day_mean_speed'].isna().sum()},  Test Nan : {test['day_mean_speed'].isna().sum()}")
print(f"Trains Nan : {train['road_mean_speed'].isna().sum()},  Test Nan : {test['road_mean_speed'].isna().sum()}")

# (차선수, 통행속도제한, 도로등급, 도로유형)별 평균속도 파생변수 추가 (4 columns)

In [ ]:
print(f"Trains : {len(train)} , Tests : {len(test)}")
lane_mean = train.groupby(['lane_count']).mean()
lane_mean.drop(columns=lane_mean.columns.drop('target'), inplace=True)
lane_mean.rename(columns={'target':'lane_mean_speed'},inplace=True)

maxspeed_mean = train.groupby(['maximum_speed_limit']).mean()
maxspeed_mean.drop(columns=maxspeed_mean.columns.drop('target'), inplace=True)
maxspeed_mean.rename(columns={'target':'maxspeed_mean'},inplace=True)

roadrating_mean = train.groupby(['road_rating']).mean()
roadrating_mean.drop(columns=roadrating_mean.columns.drop('target'), inplace=True)
roadrating_mean.rename(columns={'target':'roadrating_mean'},inplace=True)

roadtype_mean = train.groupby(['road_type']).mean()
roadtype_mean.drop(columns=roadtype_mean.columns.drop('target'), inplace=True)
roadtype_mean.rename(columns={'target':'roadtype_mean'},inplace=True)

train = pd.merge(train,lane_mean,how="left",on=['lane_count'])
train = pd.merge(train,maxspeed_mean,how="left",on=['maximum_speed_limit'])
train = pd.merge(train,roadrating_mean,how="left",on=['road_rating'])
train = pd.merge(train,roadtype_mean,how="left",on=['road_type'])

test = pd.merge(test,lane_mean,how="left",on=['lane_count'])
test = pd.merge(test,maxspeed_mean,how="left",on=['maximum_speed_limit'])
test = pd.merge(test,roadrating_mean,how="left",on=['road_rating'])
test = pd.merge(test,roadtype_mean,how="left",on=['road_type'])

print(f"Trains : {len(train)} , Tests : {len(test)}")
print(f"Trains Nan : {train['lane_count'].isna().sum()},  Test Nan : {test['lane_count'].isna().sum()}")
print(f"Trains Nan : {train['maximum_speed_limit'].isna().sum()},  Test Nan : {test['maximum_speed_limit'].isna().sum()}")
print(f"Trains Nan : {train['road_rating'].isna().sum()},  Test Nan : {test['road_rating'].isna().sum()}")
print(f"Trains Nan : {train['road_type'].isna().sum()},  Test Nan : {test['road_type'].isna().sum()}")

# 7월 (요일별, 시간별, 도로별, 차선수, 통행속도제한, 도로등급, 도로유형)별 평균속도 파생변수 추가

In [1]:
import math
tqdm.pandas()
train_july=train
train_july['month'] = train_july.progress_apply(lambda x : int(str(x['base_date'])[4:6]),axis=1)
train_july=train_july[train_july['month']==7].drop(columns=['month'])

print(f"Trains : {len(train)} , Tests : {len(test)}")
hour_mean = train_july.groupby(['base_hour']).mean()
hour_mean.drop(columns=hour_mean.columns.drop('target'), inplace=True)
hour_mean.rename(columns={'target':'hour_mean_july'},inplace=True)

day_mean = train_july.groupby(['day_of_week']).mean()
day_mean.drop(columns=day_mean.columns.drop('target'), inplace=True)
day_mean.rename(columns={'target':'day_mean_july'},inplace=True)

road_mean = train_july.groupby(['road_name']).mean()
road_mean.drop(columns=road_mean.columns.drop('target'), inplace=True)
road_mean.rename(columns={'target':'road_mean_july'},inplace=True)

lane_mean = train_july.groupby(['lane_count']).mean()
lane_mean.drop(columns=lane_mean.columns.drop('target'), inplace=True)
lane_mean.rename(columns={'target':'lane_mean_july'},inplace=True)

maxspeed_mean = train_july.groupby(['maximum_speed_limit']).mean()
maxspeed_mean.drop(columns=maxspeed_mean.columns.drop('target'), inplace=True)
maxspeed_mean.rename(columns={'target':'maxspeed_mean_july'},inplace=True)

roadrating_mean = train_july.groupby(['road_rating']).mean()
roadrating_mean.drop(columns=roadrating_mean.columns.drop('target'), inplace=True)
roadrating_mean.rename(columns={'target':'roadrating_mean_july'},inplace=True)

roadtype_mean = train_july.groupby(['road_type']).mean()
roadtype_mean.drop(columns=roadtype_mean.columns.drop('target'), inplace=True)
roadtype_mean.rename(columns={'target':'roadtype_mean_july'},inplace=True)

train = pd.merge(train,hour_mean,how="left",on=['base_hour'])
train = pd.merge(train,day_mean,how="left",on=['day_of_week'])
train = pd.merge(train,road_mean,how="left",on=['road_name'])

train = pd.merge(train,lane_mean,how="left",on=['lane_count'])
train = pd.merge(train,maxspeed_mean,how="left",on=['maximum_speed_limit'])
train = pd.merge(train,roadrating_mean,how="left",on=['road_rating'])
train = pd.merge(train,roadtype_mean,how="left",on=['road_type'])

test = pd.merge(test,hour_mean,how="left",on=['base_hour'])
test = pd.merge(test,day_mean,how="left",on=['day_of_week'])
test = pd.merge(test,road_mean,how="left",on=['road_name'])

test = pd.merge(test,lane_mean,how="left",on=['lane_count'])
test = pd.merge(test,maxspeed_mean,how="left",on=['maximum_speed_limit'])
test = pd.merge(test,roadrating_mean,how="left",on=['road_rating'])
test = pd.merge(test,roadtype_mean,how="left",on=['road_type'])

# 7월 데이터에 없는 도로 평균속도 column 전체기간 도로 평균속도로 대체 
train['road_mean_july'] = train.progress_apply(lambda row : row['road_mean_speed'] if math.isnan(row['road_mean_july']) else row['road_mean_july'],axis=1)

train.drop(columns=['month'],inplace=True)
print(f"Trains : {len(train)} , Tests : {len(test)}")
print(f"Trains Nan : {train['hour_mean_july'].isna().sum()},  Test Nan : {test['hour_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['day_mean_july'].isna().sum()},  Test Nan : {test['day_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['road_mean_july'].isna().sum()},  Test Nan : {test['road_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['lane_mean_july'].isna().sum()},  Test Nan : {test['lane_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['maxspeed_mean_july'].isna().sum()},  Test Nan : {test['maxspeed_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['roadrating_mean_july'].isna().sum()},  Test Nan : {test['roadrating_mean_july'].isna().sum()}")
print(f"Trains Nan : {train['roadtype_mean_july'].isna().sum()},  Test Nan : {test['roadtype_mean_july'].isna().sum()}")

In [ ]:
train.to_parquet('./dataset/train_파생변수.parquet',index=False)
test.to_parquet('./dataset/test_파생변수.parquet',index=False)